In [1]:
import pandas as pd



In [2]:

df = pd.read_csv('/workspaces/vikrant_dubai/final/comb/AJ_combined.csv')
unique_station_names = df['Station Name'].unique()


/tmp/ipykernel_27363/1962484940.py:1: DtypeWarning: Columns (6,9,15,23,27,29,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspaces/vikrant_dubai/final/comb/AJ_combined.csv')


In [3]:
unique_station_names

array(['Masfout', 'Envi', 'AjmanX', 'Manama', 'Jurf'], dtype=object)

In [9]:
def create_finder_table(df):
    df['vlookup'] = df['Station Name'].str[:3] + df['Date'].str.replace('/', '')
    df_grouped = df.groupby(['Station Name', 'Date']).first().reset_index()
    df_grouped['vlookup'] = df_grouped['Station Name'].str[:5] + df_grouped['Date'].str.replace('/', '')
    Finder_table = df_grouped[['Station Name', 'Date', 'vlookup']]
    return Finder_table

# Example usage:
Finder_table = create_finder_table(df)
Finder_table.to_csv('/workspaces/vikrant_dubai/SQL/Finder_table.csv', index=False)

In [5]:


pollutants = ['SO2', 'NO2','CO','PM10', 'PM2.5']

# Convert pollutant columns to numeric, forcing errors to NaN
df[pollutants] = df[pollutants].apply(pd.to_numeric, errors='coerce')

avg_values = df.groupby(['Station Name', 'Date'])[pollutants].mean().reset_index()

final_df = pd.merge(avg_values, Finder_table, on=['Station Name', 'Date'], suffixes=(' Avg 24H', ''))
final_df.rename(columns={'Station Name': 'Name'}, inplace=True)
final_df['vlookup'] = final_df['Name'].str[:5] + final_df['Date'].str.replace('/', '')
final_df=final_df.drop(columns=['Name', 'Date'])
final_df



,SO2,NO2,CO,PM10,PM2.5,vlookup
0,1.908333,28.313750,1.162500,1.059583,0.085417,Ajman01012023
1,1.296667,26.647083,1.319167,23.330833,7.953750,Ajman01022023
2,2.155000,29.250417,0.584583,17.247500,8.151667,Ajman01032023
3,2.024583,17.529167,0.818333,24.255000,9.757917,Ajman01042023
4,2.935417,33.590833,0.326667,NaN,NaN,Ajman01052023
...,...,...,...,...,...,...
1820,3.886333,4.394500,0.162237,79.647500,33.690417,Masfo31052023
1821,5.227808,14.374129,0.216667,45.990417,18.035417,Masfo31072023
1822,6.245158,10.898500,0.202500,43.170833,19.293333,Masfo31082023
1823,4.120258,17.298975,0.150417,15.979583,6.572917,Masfo31102023


In [ ]:

# Select relevant columns
selected_columns = df[['Station Name', 'Date', 'O3']].copy()

# Convert 'O3' column to numeric type
selected_columns['O3'] = pd.to_numeric(selected_columns['O3'], errors='coerce')

# Convert 'Date' column to datetime format
selected_columns['Date'] = pd.to_datetime(selected_columns['Date'], dayfirst=True)

# Extract unique station names
unique_station_names = selected_columns["Station Name"].unique()

# Function to process data for each station
def process_data(values, dates):
    df_temp = pd.DataFrame({'Index': range(len(values)), 'Values': values, 'Date': dates})

    # Compute 8-hour moving average
    df_temp['8h_Avg'] = df_temp['Values'].rolling(window=8, min_periods=1).mean()

    # Compute Data Completeness (datacap)
    df_temp['datacap'] = df_temp['Values'].rolling(window=8, min_periods=1).count() / 8 * 100

    # Function to calculate adjusted averages
    def calculate_adjusted_avg(df_temp, max_threshold):
        adjusted_avg = pd.Series(index=df_temp.index, dtype=float)  # Initialize with NaN
    
        # Condition 1: Where 'datacap' >= 75, use '8h_Avg'
        adjusted_avg[df_temp['datacap'] >= 75] = df_temp['8h_Avg']

        # Condition 2: If 'datacap' < 75, check max(rolling 8h_Avg) > 100
        low_data_mask = df_temp['datacap'] < 75  # Identify low datacap rows
        rolling_max_8h_Avg = df_temp['8h_Avg'].rolling(window=8, min_periods=1).max()

        # Apply only if rolling 8h_Avg max exceeds 100
        adjusted_avg[low_data_mask & (rolling_max_8h_Avg > max_threshold)] = rolling_max_8h_Avg
        return adjusted_avg

    # Compute adjusted averages
    df_temp['8h_Avg_WHO'] = calculate_adjusted_avg(df_temp, 100)
    df_temp['8h_Avg_NS'] = calculate_adjusted_avg(df_temp, 120)

    return df_temp

# Process data for each station
dfs = []
for station in unique_station_names:
    station_data = selected_columns[selected_columns['Station Name'] == station].copy()
    station_values = station_data["O3"].tolist()
    station_dates = station_data["Date"].tolist()

    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all processed DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange and rename columns
df2 = df2[['Station Name', 'Date', 'Values', 'datacap', '8h_Avg_WHO', '8h_Avg_NS']]
df2.rename(columns={'Values': 'O3'}, inplace=True)

# Calculate 8h_Avg_WHO_DC and 8h_Avg_NS_DC per station and date
df2['8h_Avg_WHO_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_WHO'].transform(lambda x: x.notna().sum() / 24 * 100)
df2['8h_Avg_NS_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_NS'].transform(lambda x: x.notna().sum() / 24 * 100)




In [13]:
df2

,Station Name,Date,O3,datacap,8h_Avg_WHO,8h_Avg_NS,8h_Avg_WHO_DC,8h_Avg_NS_DC
0,Masfout,2023-01-01,76.5648,12.5,76.564800,76.564800,100.0,100.0
1,Masfout,2023-01-01,72.6384,25.0,74.601600,74.601600,100.0,100.0
2,Masfout,2023-01-01,66.7488,37.5,71.984000,71.984000,100.0,100.0
3,Masfout,2023-01-01,54.9696,50.0,67.730400,67.730400,100.0,100.0
4,Masfout,2023-01-01,72.6384,62.5,68.712000,68.712000,100.0,100.0
...,...,...,...,...,...,...,...,...
43817,Jurf,2023-12-31,39.1820,100.0,75.781250,75.781250,100.0,100.0
43818,Jurf,2023-12-31,20.0220,100.0,67.806625,67.806625,100.0,100.0
43819,Jurf,2023-12-31,2.5770,100.0,57.101625,57.101625,100.0,100.0
43820,Jurf,2023-12-31,0.3210,100.0,46.065875,46.065875,100.0,100.0


In [17]:

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['8h_Avg_WHO_DC', '8h_Avg_NS_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_WHO'] = None
df2['Daily_Avg_8h_NS'] = None

# Iterate over each group to calculate the averages where count > 75
for (station, date), group in grouped:
    if group['8h_Avg_WHO_DC'].sum() > 75:  # Check if total count > 75 for WHO
        avg_value_who = group['8h_Avg_WHO'].mean()  # Calculate average for WHO
        df2.loc[group.index[-1], 'Daily_Avg_8h_WHO'] = avg_value_who  # Assign to last row
    
    if group['8h_Avg_NS_DC'].sum() > 75:  # Check if total count > 75 for NS
        avg_value_ns = group['8h_Avg_NS'].mean()  # Calculate average for NS
        df2.loc[group.index[-1], 'Daily_Avg_8h_NS'] = avg_value_ns  # Assign to last row



# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station = last_rows_per_date_station[['vlookup'] + [col for col in last_rows_per_date_station.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station.reset_index(drop=True, inplace=True)

last_rows_per_date_station



,vlookup,O3,datacap,8h_Avg_WHO,8h_Avg_NS,8h_Avg_WHO_DC,8h_Avg_NS_DC,Daily_Avg_8h_WHO,Daily_Avg_8h_NS
0,Masfo01012023,109.9392,100.0,127.36260,127.36260,79.166667,79.166667,104.587758,104.587758
1,Masfo02012023,96.1968,100.0,117.79200,117.79200,100.000000,100.000000,102.648775,102.648775
2,Masfo03012023,104.0496,100.0,117.79200,117.79200,100.000000,100.000000,112.832875,112.832875
3,Masfo04012023,109.9392,100.0,118.03740,118.03740,100.000000,100.000000,113.896275,113.896275
4,Masfo05012023,86.3808,100.0,108.95760,108.95760,100.000000,100.000000,107.659025,107.659025
...,...,...,...,...,...,...,...,...,...
1821,Jurf27122023,1.1170,100.0,27.09300,27.09300,58.333333,58.333333,None,None
1822,Jurf28122023,1.0870,100.0,37.70175,37.70175,100.000000,100.000000,32.031406,32.031406
1823,Jurf29122023,0.6570,100.0,32.51800,32.51800,100.000000,100.000000,33.478885,33.478885
1824,Jurf30122023,1.5540,100.0,55.60650,55.60650,100.000000,100.000000,37.769464,37.769464


In [ ]:



selected_columns = df[['Station Name', 'Date', 'CO']]

df=selected_columns

# Convert 'Date' column to datetime format for proper processing
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Extract unique station names
unique_station_names = df["Station Name"].unique()

# Define the process_data function for CO transformation
def process_data(values, dates):
    # Create DataFrame
    df_temp = pd.DataFrame({'Index': range(len(values)), 'CO': values, 'Date': dates})

    # Calculate 8-hour moving average
    df_temp['C0_8h_Avg'] = df_temp['CO'].rolling(window=8).mean()

    # Calculate datacap (percentage of available data in 8-hour window)
    df_temp['datacap'] = df_temp['CO'].rolling(window=8).count() / 8 * 100

    # Ensure first 7 values of 'datacap' are blank (NaN)
    df_temp.loc[:6, 'datacap'] = None

    return df_temp

# Process data for each station name
dfs = []
for station in unique_station_names:
    station_data = df[df['Station Name'] == station]
    station_values = station_data["CO"].tolist()
    station_dates = station_data["Date"].tolist()
    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange the columns as specified
df2 = df2[['Station Name', 'Date', 'CO', 'datacap', 'C0_8h_Avg']]

# Calculate C0_8h_Avg_DC (daily data capture percentage for CO 8h Avg)
df2['C0_8h_Avg_DC'] = df2.groupby(['Station Name', 'Date'])['C0_8h_Avg'].transform(lambda x: x.count() / 24 * 100)

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['C0_8h_Avg_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_C0'] = None

# Iterate over each group to calculate the daily averages where count > 75
for (station, date), group in grouped:
    if group['C0_8h_Avg_DC'].sum() > 75:  # Check if total count > 75
        avg_value_c0 = group['C0_8h_Avg'].mean()  # Calculate average
        df2.loc[group.index[-1], 'Daily_Avg_8h_C0'] = avg_value_c0  # Assign to last row


# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station2 = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station2 = last_rows_per_date_station2[['vlookup'] + [col for col in last_rows_per_date_station2.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station2.reset_index(drop=True, inplace=True)

last_rows_per_date_station2



,vlookup,CO,datacap,C0_8h_Avg,C0_8h_Avg_DC,Daily_Avg_8h_C0
0,Masfo01102023,0.23,100.0,0.20125,70.833333,None
1,Masfo02102023,0.21,100.0,0.20250,100.000000,0.181667
2,Masfo03102023,0.19,100.0,0.17000,100.000000,0.194583
3,Masfo04102023,0.20,100.0,0.17375,100.000000,0.183073
4,Masfo05102023,0.41,100.0,0.27000,100.000000,0.183958
...,...,...,...,...,...,...
1821,Jurf27072023,0.20,100.0,0.25125,100.000000,0.192083
1822,Jurf28072023,0.31,100.0,0.26125,100.000000,0.185625
1823,Jurf29072023,0.22,100.0,0.26875,100.000000,0.216042
1824,Jurf30072023,0.20,100.0,0.37875,100.000000,0.267396


In [ ]:



# Merge last_rows_per_date_station and last_rows_per_date_station2 on 'vlookup'
combined_df = pd.merge(last_rows_per_date_station, last_rows_per_date_station2, on='vlookup', suffixes=('_O3', '_CO'))

# Merge the result with final_df on 'vlookup'
combined_df = pd.merge(combined_df, final_df, on='vlookup')

# Display the combined dataframe
D24hparameter=combined_df[['vlookup','SO2', 'NO2', 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0', 'PM10', 'PM2.5']]

D24hparameter.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter.csv', index=False)




# Round all columns of D24hparameter to 1 decimal place
D24hparameter_rounded = D24hparameter.round(1)
# Convert 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', and 'Daily_Avg_8h_C0' to numeric, forcing errors to NaN
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].apply(pd.to_numeric, errors='coerce')

# Round the specified columns to 1 decimal place
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].round(1)
# Display the new table
D24hparameter_rounded.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter_rounded.csv', index=False)    





df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_NS = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_NS['SO2'] = df['SO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'NO2' column based on the condition
KPI_NS['NO2'] = df['NO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_NS['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 120.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_NS['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_NS.to_csv('/workspaces/vikrant_dubai/SQL/KPI_NS.csv', index=False)



df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_WHO = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_WHO['SO2'] = df['SO2'].apply(lambda x: 1 if x < 20.4 else 0)

# Add 'NO2' column based on the condition
KPI_WHO['NO2'] = df['NO2'].apply(lambda x: 1 if x < 75.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_WHO['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 100.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_WHO['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_WHO.to_csv('/workspaces/vikrant_dubai/SQL/KPI_WHO.csv', index=False)




KPI_PM = df[['vlookup']].copy()
KPI_PM['PM10'] = df['PM10'].apply(lambda x: 1 if x < 150.4 else 0)
KPI_PM['PM2.5'] = df['PM2.5'].apply(lambda x: 1 if x < 60.4 else 0)
KPI_PM.to_csv('/workspaces/vikrant_dubai/SQL/KPI_PM.csv', index=False)